In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._


//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//Circe para trabajar con JSON
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._


//Sttp para realizar request
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._
import sttp.client.basicRequest

//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                  



import org.apache.spark.sql._

import org.apache.spark.sql.functions._


//Reducir numero logs

import org.apache.log4j.{Level, Logger}

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._


//Sttp para realizar request

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._

import sttp.client.basicRequest

//For adding extra dependenies

import $ivy.$                               

//Plotly

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

### Inicio Spark Session y Algunos métodos para descargar los datos

In [3]:
//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

//Crear URL
def createUri(category: String, widget: String, start_date: String, end_date: String, time_trunc: String, lang: String, 
              geo_trunc: String = "", geo_limit: String = "", geo_ids: String = ""): String = {
  val baseUrl = s"https://apidatos.ree.es/$lang/datos/$category/$widget"
  
  // Crear un mapa con los parámetros obligatorios y opcionales
  val params = Map(
    "start_date" -> start_date,
    "end_date" -> end_date,
    "time_trunc" -> time_trunc,
    "geo_trunc" -> geo_trunc,
    "geo_limit" -> geo_limit,
    "geo_ids" -> geo_ids
  ).filter { case (_, v) => v.nonEmpty } // Filtrar los parámetros que no están vacíos

  //Especifico el orden concreto de los paramtros ya que, Scala map no te garantiza el orden
  val orderedKeys = Seq("start_date", "end_date", "time_trunc", "geo_trunc", "geo_limit", "geo_ids")

  // Unir los parámetros en una cadena
  val queryString = orderedKeys.flatMap(key => params.get(key).map(value => s"$key=$value")).mkString("&")
  
  s"$baseUrl?$queryString"
}

//Lanzar llamada y obtener guardar datos
def getDataApi(apiUrl: String): Either[String, String] = {
 
    val newRequest = basicRequest.get(uri"$apiUrl")
    val response = newRequest.send()
  
  response.body match {
    case Right(body) => 
      Right(body)
      
    case Left(error) => 
      Left(s"Error fetching data: $error")
  }
}

// Transformar response a DataFrame
def responseToDF(response: Either[String, String]): DataFrame = {
    response match {
    case Right(body) => 
        val rdd = spark.sparkContext.parallelize(Seq(body))
        spark.read.json(rdd)
    case Left(error) => 
        println(error)
        spark.emptyDataFrame
    }
}



Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@6cae07e0
import spark.implicits._

//Crear URL

defined function createUri
defined function getDataApi
defined function responseToDF

# Modelo Balance-electrico
Datos diarios del balance electrico a nivel nacional

In [4]:
val category = "balance"
val widget = "balance-electrico"
val start_date = "2024-01-01T00:00"
val end_date = "2024-01-31T23:59"
val time_trunc = "day"
val lang = "es"
val geo_trunc = "electric_system"
val geo_limit = "ccaa"
val geo_ids = "13"

//Creación Uri
val uriBalance = createUri(category, widget, start_date, end_date, time_trunc, lang) //Generacion Nacional

//Hacemos la request a la api REData y transformamos en un dataframe
val dFrameBalance = responseToDF(getDataApi(uriBalance))

json at cmd2.sc:52

16 / 16

category: String = "balance"
widget: String = "balance-electrico"
start_date: String = "2024-01-01T00:00"
end_date: String = "2024-01-31T23:59"
time_trunc: String = "day"
lang: String = "es"
geo_trunc: String = "electric_system"
geo_limit: String = "ccaa"
geo_ids: String = "13"
uriBalance: String = "https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2024-01-01T00:00&end_date=2024-01-31T23:59&time_trunc=day"
dFrameBalance: DataFrame = [data: struct<attributes: struct<description: string, last-update: string ... 1 more field>, id: string ... 2 more fields>, included: array<struct<attributes:struct<content:array<struct<attributes:struct<color:string,composite:boolean,description:string,last-update:string,magnitude:string,title:string,total:double,type:string,values:array<struct<datetime:string,percentage:double,value:double>>>,groupId:string,id:string,type:string>>,description:string,last-update:string,magnitude:string,title:string>,id:string,type:string>>]

In [6]:
dFrameBalance.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- attributes: struct (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- last-update: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- meta: struct (nullable = true)
 |    |    |-- cache-control: struct (nullable = true)
 |    |    |    |-- cache: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- included: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attributes: struct (nullable = true)
 |    |    |    |-- content: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- attributes: struct (nullable = true)
 |    |    |    |    |    |    |-- color: string (nullable = true)
 |    |    |    |    |    |    |-- composite: boolean (nullable = true)
 |    |    |    |    |    |    |-- description: string (nullable = tru

#### Transformaciones necesarias para desanidar respuesta y creación de un modelo manipulable

In [7]:
//Crearemos una tabla con las columnas Familia, Tipo, Fecha, Porcentje y Valor, además agregamos una nueva columna llamada BajasEmisiones
def transformBalanceModel(df: DataFrame): DataFrame = {

        // Definir los tipos de energía de bajas emisiones
        val bajasEmisiones = Seq(
            "Solar fotovoltaica", 
            "Solar térmica", 
            "Nuclear", 
            "Hidroeólica", 
            "Eólica", 
            "Generación renovable", 
            "Otras renovables",
            "Hidráulica", 
            "Residuos renovables"
        )

        //Crear modelo desanidado
        df.withColumn("FamilyGroup", explode($"included"))
            .withColumn("TypeGroup", explode($"FamilyGroup.attributes.content"))
            .withColumn("Values", explode($"TypeGroup.attributes.values"))
            .select(
                $"FamilyGroup.type".as("Familia"),
                $"TypeGroup.type".as("Tipo"),
                $"Values.datetime".as("FechaCompleta"), 
                $"Values.percentage".as("Porcentaje"),
                $"Values.value".as("Valor")
            )
            .withColumn("FechaCompleta", $"FechaCompleta".cast("timestamp"))
            .withColumn("BajasEmisiones", $"Tipo".isin(bajasEmisiones: _*))
    }
val modelDataFrameBalance = transformBalanceModel(dFrameBalance)

defined function transformBalanceModel
modelDataFrameBalance: DataFrame = [Familia: string, Tipo: string ... 4 more fields]

#### Creamos datasetn Balance
Para ello, lo primero es crear una case class con la estructura necesaria.

Hay que añadir "org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)" en la misma celda donde defines las clases case implicadas para evitar un error


In [8]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this);
case class Balance(
    Familia: String,
    Tipo: String,
    FechaCompleta: java.sql.Timestamp,
    Valor: Double,
    Porcentaje: Double,
    Dia: String,
    Hora: String,
    BajasEmisiones: Boolean
)

val modelDatasetBalance: Dataset[Balance] = modelDataFrameBalance.as[Balance]
//val dSetBalance : Dataset[Balance] = modelDataFrame.collect.toSeq.toDs()


: 

#### Guardar dataset en un archivo formato .csv

In [7]:
modelDatasetBalance.coalesce(1).write
  .option("header", "true")
  .csv("data/modelosCsv/datasetCoalesce_balance.csv")

: 

In [8]:
// based on a snippet by Ivan Zaitsev
// https://github.com/almond-sh/almond/issues/180#issuecomment-364711999
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        val str = cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
        if (truncate > 0 && str.length > truncate) {
          // do not show ellipses for strings shorter than 4 characters.
          if (truncate < 4) str.substring(0, truncate)
          else str.substring(0, truncate - 3) + "..."
        } else {
          str
        }
      }: Seq[String]
    }

    publish.html(s"""
      <table class="table">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

defined class RichDF

In [9]:
modelDatasetBalance.toDF.showHTML()

take at cmd7.sc:4

1 / 1

take at cmd7.sc:4

4 / 4

take at cmd7.sc:4

11 / 11

Familia,Tipo,FechaCompleta,Porcentaje,Valor,Dia,Hora,BajasEmisiones
Renovable,Hidráulica,2022-12-31 23:00:...,0.2721558942556826,76595.584,2022-12-31,23:00:00,true
Renovable,Hidráulica,2023-01-01 23:00:...,0.4915900251582455,136518.071,2023-01-01,23:00:00,true
Renovable,Hidráulica,2023-01-02 23:00:...,0.516836233628069,153711.167,2023-01-02,23:00:00,true
Renovable,Hidráulica,2023-01-03 23:00:...,0.48751377909460486,138220.83,2023-01-03,23:00:00,true
Renovable,Hidráulica,2023-01-04 23:00:...,0.5385302584610008,143184.392,2023-01-04,23:00:00,true
Renovable,Hidráulica,2023-01-05 23:00:...,0.45128187316793955,131153.006,2023-01-05,23:00:00,true
Renovable,Hidráulica,2023-01-06 23:00:...,0.17416824791827182,74548.891,2023-01-06,23:00:00,true
Renovable,Hidráulica,2023-01-07 23:00:...,0.16741886839924458,80976.023,2023-01-07,23:00:00,true
Renovable,Hidráulica,2023-01-08 23:00:...,0.2187615393081991,102267.435,2023-01-08,23:00:00,true
Renovable,Hidráulica,2023-01-09 23:00:...,0.3521618890281243,148895.976,2023-01-09,23:00:00,true


# Modelo Mercados

Para este modelo vamos a utilizar los datos de la categoria Mercados y el widget "precios-mercados-tiempo-real" que nos da el precio de la energía cada hora, 
el response nos da dos tipos de precio, el PVPC y el Precio mercado spot, los dos en (€/MWh)

In [10]:
val category = "mercados"
val widget = "precios-mercados-tiempo-real"
val start_date = "2024-01-01T00:00"
val end_date = "2024-01-31T23:59"
val time_trunc = "hour"
val lang = "es"
val geo_limits = "ccaa"
val ids = "13"

val uriMercadosTR= createUri(category, widget, start_date, end_date, time_trunc, lang)

//Hacemos la request a la api REData y transformamos en un dataframe
val dFrameMercadosTR = responseToDF(getDataApi(uriMercadosTR))

json at cmd2.sc:52

16 / 16

category: String = "mercados"
widget: String = "precios-mercados-tiempo-real"
start_date: String = "2024-01-01T00:00"
end_date: String = "2024-01-31T23:59"
time_trunc: String = "hour"
lang: String = "es"
geo_limits: String = "ccaa"
ids: String = "13"
uriMercadosTR: String = "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real?start_date=2024-01-01T00:00&end_date=2024-01-31T23:59&time_trunc=hour"
dFrameMercadosTR: DataFrame = [data: struct<attributes: struct<description: string, last-update: string ... 1 more field>, id: string ... 2 more fields>, included: array<struct<attributes:struct<color:string,composite:boolean,description:string,last-update:string,magnitude:string,title:string,type:string,values:array<struct<datetime:string,percentage:double,value:double>>>,groupId:string,id:string,type:string>>]

#### Transformaciones para modelar los datos
Después de desanidar el json también realizamos un pivot sobre la columna TipoMercado para que por cada Fecha tengamos 2 valores de mercado, el PVPC y el Spot

In [11]:
// Desanidar y transformar el DataFrame
val transformedDF = dFrameMercadosTR
  .withColumn("included", explode($"included"))
  .withColumn("TipoMercado", $"included.type")
  .withColumn("Values", explode($"included.attributes.values"))
  .select(
    $"Values.datetime".cast("timestamp").as("Fecha"),
    $"TipoMercado",
    $"Values.value".as("Valor"),
    $"Values.percentage".as("Porcentaje")
  )
// Pivotar el DataFrame para obtener las columnas necesarias
val pivotedDF = transformedDF
    .groupBy("Fecha")
    .pivot("TipoMercado")
    .agg(
    first("Valor").as("Valor"),
    first("Porcentaje").as("Porcentaje")
    )
    .withColumnRenamed("PVPC (€/MWh)_Valor", "Valor_PVPC")
    .withColumnRenamed("PVPC (€/MWh)_Porcentaje", "Porcentaje_PVPC")
    .withColumnRenamed("Precio mercado spot (€/MWh)_Valor", "Valor_Mercado_Spot")
    .withColumnRenamed("Precio mercado spot (€/MWh)_Porcentaje", "Porcentaje_Mercado_Spot")

pivot at cmd10.sc:14

16 / 16

pivot at cmd10.sc:14

200 / 200

pivot at cmd10.sc:14

1 / 1

transformedDF: DataFrame = [Fecha: timestamp, TipoMercado: string ... 2 more fields]
pivotedDF: DataFrame = [Fecha: timestamp, Valor_PVPC: double ... 3 more fields]

In [12]:
pivotedDF.show()

show at cmd11.sc:1

16 / 16

show at cmd11.sc:1

200 / 200

show at cmd11.sc:1

...

show at cmd11.sc:1

4 / 4

show at cmd11.sc:1

2 / 2

+-------------------+----------+------------------+------------------+-----------------------+
|              Fecha|Valor_PVPC|   Porcentaje_PVPC|Valor_Mercado_Spot|Porcentaje_Mercado_Spot|
+-------------------+----------+------------------+------------------+-----------------------+
|2024-01-29 03:00:00|    101.89|0.6639082556851502|             51.58|     0.3360917443148498|
|2024-01-09 01:00:00|    137.24|0.5823397123095855|             98.43|     0.4176602876904146|
|2024-01-01 18:00:00|    103.34|0.6527698818773293|             54.97|     0.3472301181226707|
|2024-01-26 21:00:00|    140.75|0.6524359152644509|             74.98|    0.34756408473554906|
|2024-01-19 11:00:00|    192.91|0.7101417264862876|             78.74|     0.2898582735137125|
|2024-01-10 11:00:00|    206.88|0.6741616971355948|             99.99|     0.3258383028644051|
|2024-01-15 14:00:00|    155.09|0.6325298747909784|              90.1|    0.36747012520902156|
|2024-01-29 08:00:00|    151.57|0.6316469411568595

#### Creamos dataset MercadosPorHora
Recordar añadir "org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)" en la misma celda donde defines las clases case implicadas para evitar un error


In [13]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class MercadoPorHora(
  Fecha: java.sql.Timestamp,
  Valor_PVPC: Double,
  Porcentaje_PVPC: Double,
  Valor_Mercado_Spot: Double,
  Porcentaje_Mercado_Spot: Double
)

val dsetMercadoPorHora: Dataset[MercadoPorHora] = pivotedDF.as[MercadoPorHora]

defined class MercadoPorHora
dsetMercadoPorHora: Dataset[MercadoPorHora] = [Fecha: timestamp, Valor_PVPC: double ... 3 more fields]

In [14]:
dsetMercadoPorHora.show()

show at cmd13.sc:1

16 / 16

show at cmd13.sc:1

200 / 200

show at cmd13.sc:1

...

show at cmd13.sc:1

4 / 4

show at cmd13.sc:1

2 / 2

+-------------------+----------+------------------+------------------+-----------------------+
|              Fecha|Valor_PVPC|   Porcentaje_PVPC|Valor_Mercado_Spot|Porcentaje_Mercado_Spot|
+-------------------+----------+------------------+------------------+-----------------------+
|2024-01-29 03:00:00|    101.89|0.6639082556851502|             51.58|     0.3360917443148498|
|2024-01-09 01:00:00|    137.24|0.5823397123095855|             98.43|     0.4176602876904146|
|2024-01-01 18:00:00|    103.34|0.6527698818773293|             54.97|     0.3472301181226707|
|2024-01-26 21:00:00|    140.75|0.6524359152644509|             74.98|    0.34756408473554906|
|2024-01-19 11:00:00|    192.91|0.7101417264862876|             78.74|     0.2898582735137125|
|2024-01-10 11:00:00|    206.88|0.6741616971355948|             99.99|     0.3258383028644051|
|2024-01-15 14:00:00|    155.09|0.6325298747909784|              90.1|    0.36747012520902156|
|2024-01-29 08:00:00|    151.57|0.6316469411568595

# Modelo Demanda por hora

Categoria: demanda, widget: demanda-tiempo-real
Esta response nos da valor de la demanda real de energía cada 5 min.

In [15]:
val category = "demanda"
val widget = "demanda-tiempo-real"
val start_date = "2014-01-01T00:00"
val end_date = "2014-01-31T23:59"
val time_trunc = "hour"
val lang = "es"
//val geo_limits = "ccaa"
//val ids = "13"

val uriDemanda= createUri(category, widget, start_date, end_date, time_trunc, lang)

//Hacemos la request a la api REData y transformamos en un dataframe
val dFrameDemanda = responseToDF(getDataApi(uriDemanda))

json at cmd2.sc:52

16 / 16

category: String = "demanda"
widget: String = "demanda-tiempo-real"
start_date: String = "2014-01-01T00:00"
end_date: String = "2014-01-31T23:59"
time_trunc: String = "hour"
lang: String = "es"
uriDemanda: String = "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real?start_date=2014-01-01T00:00&end_date=2014-01-31T23:59&time_trunc=hour"
dFrameDemanda: DataFrame = [data: struct<attributes: struct<description: string, last-update: string ... 1 more field>, id: string ... 2 more fields>, included: array<struct<attributes:struct<color:string,composite:boolean,description:string,last-update:string,magnitude:string,title:string,type:string,values:array<struct<datetime:string,percentage:double,value:bigint>>>,groupId:string,id:string,type:string>>]

#### Transformaciones para modelar los datos de demanda
Al igual que con el mercado aqui vamos a realizar otro pivot para desglosar la demanda en 3 columnas: Demanda real, Demanda programada y Demanda prevista

In [16]:
// Desanidar y transformar el DataFrame
val transformedDfDemanda = dFrameDemanda
    .withColumn("included", explode($"included"))
    .withColumn("TipoDemanda", $"included.type")
    .withColumn("Values", explode($"included.attributes.values"))
    .select(
    $"Values.datetime".cast("timestamp").as("Fecha"),
    $"TipoDemanda",
    $"Values.value".as("Valor"),
    $"Values.percentage".as("Porcentaje")
  )
// Pivotar el DataFrame para obtener las columnas necesarias
val pivotedDfDemanda = transformedDfDemanda
    .groupBy("Fecha")
    .pivot("TipoDemanda")
    .agg(
    first("Valor").as("Valor"),
    first("Porcentaje").as("Porcentaje")
    )
    .withColumnRenamed("Demanda prevista_Valor", "Valor_previsto")
    .withColumnRenamed("Demanda prevista_Porcentaje", "Porcentaje_previsto")
    .withColumnRenamed("Demanda programada_Valor", "Valor_programado")
    .withColumnRenamed("Demanda programada_Porcentaje", "Porcentaje_programado")
    .withColumnRenamed("Demanda real_Valor", "Valor_real")
    .withColumnRenamed("Demanda real_Porcentaje", "Porcentaje_real")


pivot at cmd15.sc:14

16 / 16

pivot at cmd15.sc:14

200 / 200

transformedDfDemanda: DataFrame = [Fecha: timestamp, TipoDemanda: string ... 2 more fields]
pivotedDfDemanda: DataFrame = [Fecha: timestamp, Valor_previsto: bigint ... 5 more fields]

In [17]:
pivotedDfDemanda.show()

show at cmd16.sc:1

16 / 16

show at cmd16.sc:1

200 / 200

show at cmd16.sc:1

1 / 1

show at cmd16.sc:1

1 / 1

+-------------------+--------------+-------------------+----------------+---------------------+----------+-------------------+
|              Fecha|Valor_previsto|Porcentaje_previsto|Valor_programado|Porcentaje_programado|Valor_real|    Porcentaje_real|
+-------------------+--------------+-------------------+----------------+---------------------+----------+-------------------+
|2014-01-07 15:10:00|         31859| 0.3346779700188039|           31609|   0.3320517264924942|     31725| 0.3332703034887019|
|2014-01-29 09:40:00|         36354| 0.3343480700076335|           36310|   0.3339434016057978|     36067|0.33170852838656867|
|2014-01-21 17:50:00|         37367|0.33383363261951343|           36671|   0.3276156272055605|     37895| 0.3385507401749261|
|2014-01-09 20:40:00|         35264| 0.3328959416978977|           35732|  0.33731391188603904|     34935| 0.3297901464160633|
|2014-01-02 22:20:00|         28422|0.33310674605034923|           27940|  0.32745769068491865|     28962|0.339

In [18]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class DemandaPorHora(
  Fecha: java.sql.Timestamp,
  Valor_previsto: Double,
  Porcentaje_previsto: Double,
  Valor_programado: Double,
  Porcentaje_programado: Double,
  Valor_real: Double,
  Porcentaje_real: Double
)

val dsetDemanda: Dataset[DemandaPorHora] = pivotedDfDemanda.as[DemandaPorHora]

defined class DemandaPorHora
dsetDemanda: Dataset[DemandaPorHora] = [Fecha: timestamp, Valor_previsto: bigint ... 5 more fields]

In [19]:
dsetDemanda.show()

show at cmd18.sc:1

16 / 16

show at cmd18.sc:1

200 / 200

show at cmd18.sc:1

1 / 1

show at cmd18.sc:1

1 / 1

+-------------------+--------------+-------------------+----------------+---------------------+----------+-------------------+
|              Fecha|Valor_previsto|Porcentaje_previsto|Valor_programado|Porcentaje_programado|Valor_real|    Porcentaje_real|
+-------------------+--------------+-------------------+----------------+---------------------+----------+-------------------+
|2014-01-07 15:10:00|         31859| 0.3346779700188039|           31609|   0.3320517264924942|     31725| 0.3332703034887019|
|2014-01-29 09:40:00|         36354| 0.3343480700076335|           36310|   0.3339434016057978|     36067|0.33170852838656867|
|2014-01-21 17:50:00|         37367|0.33383363261951343|           36671|   0.3276156272055605|     37895| 0.3385507401749261|
|2014-01-09 20:40:00|         35264| 0.3328959416978977|           35732|  0.33731391188603904|     34935| 0.3297901464160633|
|2014-01-02 22:20:00|         28422|0.33310674605034923|           27940|  0.32745769068491865|     28962|0.339

In [19]:
dsetDemanda.Valor_previsto

cmd19.sc:1: value Valor_previsto is not a member of org.apache.spark.sql.Dataset[cmd19.this.cmd17.DemandaPorHora]
val res19 = dsetDemanda.Valor_previsto
                        ^Compilation Failed

: 